In [ ]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time

In [ ]:
#Stanting timming process
start = time.time()

In [ ]:
# saving the main directory
main_directory = os.getcwd()

In [ ]:
# subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]
subfolder_with_path = ["" for i in range(2)]
subfolder_with_path[0] = main_directory + '/'+'01cluster_files'
subfolder_with_path[1] = main_directory + '/'+'zzz_input_and_reference_files'

In [ ]:
###############################################################################################
# Importing information from "intensisties template" and "brain gubre map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])

# Reading "gubra_intensities_template.csv" file
gubra_intensities_template = pd.read_csv("gubra_intensities_template.csv",
                                   dtype={"IDPath": "string", 
                                          "LabelID": int, 
                                          "raw": "string", 
                                          "LabelAbrv": "string"})
# Reading "brain_allen_map.csv" file
brain_gubra_map = pd.read_csv("brain_gubra_map_fixed.csv", 
                               dtype={"IDPath": "string", "LabelAbrv": "string",
                                      "raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})


In [ ]:
# Changing directory to the "01cluster_files" folder
os.chdir(subfolder_with_path[0])

#Counting number of files on "01cluster_files" folder
list = os.listdir(subfolder_with_path[0]) 
number_files = len(list)
number_cluster_files = number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_data = [] #This array will contain df as elements

i = 0
while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    
    file = list[i]
    # Saving file's name as array
    filename_segmented = file.split('_')

    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = gubra_intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME'] = output['VOLUME'].div(1000000).round(7)
   

    # Adding a column to save hemispheres
    output['HEMISPHERE'] = ''
    output.loc[output.LabelID < 20000, 'HEMISPHERE'] = 'RIGHT'
    output.loc[output.LabelID > 20000, 'HEMISPHERE'] = 'LEFT'
        
    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('L','',1)
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('R','',1)
    
    #Adding a column with the value of the total of the found volume
    output['TOTAL_VOLUME'] = output['VOLUME'].sum()
    
    #Adding a column with the percentage of volume part form the total volume
    output['VOLUME_PERCENTAGE'] = (output['VOLUME'] /  output['TOTAL_VOLUME'] ) * 100
    
    #Adding a column with the cluster's name
    output['CLUSTER_NAME'] = 'CLUSTER_' + str(i + 1)
    
    #Adding columns with the first, second and third highest percentages
    first_three = output['VOLUME_PERCENTAGE'].nlargest(3).values
    output['FIRST_HIGHEST_VOLUME'] = output.loc[output['VOLUME_PERCENTAGE'] == first_three[0], 'raw'].item()
    output['FIRST_HIGHEST_VOL_%'] = first_three[0]
    output['SECOND_HIGHEST_VOLUME'] = output.loc[output['VOLUME_PERCENTAGE'] == first_three[1], 'raw'].item()
    output['SECOND_HIGHEST_VOL_%'] = first_three[1]
    output['THIRD_HIGHEST_VOLUME'] = output.loc[output['VOLUME_PERCENTAGE'] == first_three[2], 'raw'].item() 
    output['THIRD_HIGHEST_VOL_%'] = first_three[2]
  
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
volume_data = pd.concat(volume_data)

In [ ]:
sorted_volume_data = volume_data.filter(['CLUSTER_NAME','FIRST_HIGHEST_VOLUME','FIRST_HIGHEST_VOL_%'],
                                        ['SECOND_HIGHEST_VOLUME','SECOND_HIGHEST_VOL_%'],
                                        ['THIRD_HIGHEST_VOLUME','THIRD_HIGHEST_VOL_%'],axis=1)


In [ ]:
# Changing directory to the "01Activity_files" folder
os.chdir(subfolder_with_path[0])

#Counting number of files on "01Activity_files" folder
list = os.listdir(subfolder_with_path[0]) 
number_files = len(list)
number_activity_files = number_files

In [ ]:
#Using and timing  a loop to process all files and populate the main data-frame

activity_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

#Zeroing the counters 
counters = np.zeros(len(counters_names))

#Creating counters array to contain sample's names as elements
counters_samples = [[] for j in range(len(counters_names))]


    ##############################################
    # Innitiates loop to process each file
    ##############################################


while i < number_files:

    # Working with each file at a time
    file = list[i]
    
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]

    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file,low_memory=False)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]        

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, 
                                                   ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
#     total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)
        
    # Adding activity columns to intensitites template data-frame
    output = gubra_intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    
    #Adding a column with the sample's nickname

    counters_index = counters_names.index(output['TX_GROUP'].iloc[0] + '_' + output['marker'].iloc[0])
    if output['SAMPLE'].iloc[0] not in counters_samples[counters_index]:
        counters[counters_index] += 1
        counters_samples[counters_index].append(output['SAMPLE'].iloc[0])
        nickname = output['TX_GROUP'].iloc[0] + '_' + str(int(counters[counters_index]))

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname

    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('L','',1)
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    activity_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
activity_data = pd.concat(activity_data)

In [ ]:
#Replacing empty cells with zeros
activity_data['ACTIVITY'].fillna(0, inplace=True)

In [ ]:
#Creating density dataframe
##Merging activity, volume and volume in tissue dataframes
density = pd.merge(activity_data,
                                  volumes_data[['raw','SAMPLE','VOLUME']],
                                  on=['raw','SAMPLE'],
                                  how='left')
density = pd.merge(density,
                           volume_intissue_data[['raw','SAMPLE','VOLUME_IN_TISSUE']],
                           on=['raw','SAMPLE'],
                           how='left')
#Creating the density nickname (d_nickname)
density['DENSITY'] = density['ACTIVITY'] / density['VOLUME_IN_TISSUE']
density['d_nickname'] = 'd_' + density['nickname']


#Pivoting dataframe to have d_nicknames as columns
pivoted_density = pd.pivot_table(density, values='DENSITY', 
                                       index=['raw'],columns=['d_nickname'], aggfunc=np.sum)

In [ ]:
#Merging activity and volume ratio to filter activity where the volume ratio is below 50%
filtered_activity_data = pd.merge(activity_data,
                                  volumes_data[['raw','SAMPLE','volumes_ratio']],
                                  on=['raw','SAMPLE'],
                                  how='left')
filtered_activity_data.loc[(filtered_activity_data['volumes_ratio'] < 0.5),'ACTIVITY'] = ''

In [ ]:
#Pivoting dataframe to have nicknames as columns
pivoted_activity_data = pd.pivot_table(filtered_activity_data, values='ACTIVITY', 
                               index=['raw','hemisphere','marker'],columns=['nickname'], aggfunc=np.sum)

#Restarting index to activate'hemisphere','marker'and 'rater' as columns
pivoted_activity_data = pivoted_activity_data.reset_index()

In [ ]:
#combining all dataframes
gubra_and_activity = pd.merge(brain_gubra_map,pivoted_activity_data,on='raw',how='right')
cell_count_output = pd.merge(gubra_and_activity,pivoted_volumes_data,on='raw',how='left')
cell_count_output = pd.merge(cell_count_output,pivoted_density,on='raw',how='left')

In [ ]:
# Changing directory to the main path
os.chdir(main_directory)

In [ ]:
#creating the "cell_count_output.csv" file
cell_count_output.to_csv('GUBRA_cell_count_output.csv', index=False)

In [ ]:
#Ending timming process
end = time.time()

# Printing how many files were processed and how much time the process took
files_processed = number_activity_files = number_files+ number_volume_files + number_volintissue_files
print(files_processed,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 